In [43]:
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")

messages = [
    HumanMessage(content="List 8 wonders of the word")
]

result = llm(messages)

wonders_list = result.content.split("\n")

for wonder in wonders_list:
    print(wonder)

1. Great Wall of China
2. Petra, Jordan
3. Christ the Redeemer, Brazil
4. Taj Mahal, India
5. Machu Picchu, Peru
6. Chichen Itza, Mexico
7. Colosseum, Italy
8. Statue of Liberty, United States


### Streaming and Batching


In [21]:
for chunk in llm.stream("8 Wonders of World"):
    print(chunk.content)


1
.
 Great
 Wall
 of
 China


2
.
 Petra
,
 Jordan


3
.
 Christ
 the
 Rede
emer
,
 Brazil


4
.
 Mach
u
 Pic
chu
,
 Peru


5
.
 Ch
ichen
 It
za
,
 Mexico


6
.
 Col
os
se
um
,
 Italy


7
.
 Taj
 Mah
al
,
 India


8
.
 The
 Great
 Pyramid
 of
 G
iza
,
 Egypt



In [18]:
result = llm.batch(
    [
        "What's 2 +2 ?",
        "What's 34.9 + 23.78?"
    ]
)
for _result in result:
    print(_result.content)

2 + 2 is equal to 4.
58.68


### System & Human Message

In [16]:
from langchain.schema import SystemMessage

messages = [
    SystemMessage(content="You are Virat Kohli"),
    HumanMessage(content="Which crickrt bat manufacturer you are associated with?")
]

result = llm(messages)
result.content

'I am associated with the cricket bat manufacturer, MRF.'

### Prompt Template

In [23]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "tell me a {objective} joke about {content}"
)

filled_prompt = prompt_template.format(objective="funny", content="robots")

response = llm.invoke(filled_prompt)
response.content

'Why did the robot go on a diet?\n\nBecause it had a byte problem!'

### Chat Prompt Template

In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Defining a chat template with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful AI Mathematics teacher bot. Your name is {name}"),
        HumanMessage(content="Hello, How are you doing?"),
        AIMessage(content="I'm doing well, thanks!"),
        HumanMessage(content="{math_problem}")
    ]
)

math_problem = input("Enter Math problem:")

formatted_messages = chat_template.format_messages(name="Bob", math_problem=math_problem)

for message in formatted_messages:
    print(message.content)

You are a helpful AI Mathematics teacher bot. Your name is {name}
Hello, How are you doing?
I'm doing well, thanks!
{math_problem}


### Output Parsers

Simple Json Parser

In [52]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_parser = SimpleJsonOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a JSON object with keys 'birth_place' and 'birth_date'.
"""

json_prompt = PromptTemplate.from_template(template)

json_chain = json_prompt | llm | json_parser

input_data = {"question": "When and where were Elon Musk and Bill Gates born?"}

output = json_chain.invoke(input_data)
output

{'birth_place': {'Elon Musk': 'Pretoria, South Africa',
  'Bill Gates': 'Seattle, Washington, USA'},
 'birth_date': {'Elon Musk': 'June 28, 1971',
  'Bill Gates': 'October 28, 1955'}}

In [56]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a comma-separated list with the format: "Country, Capital, Independence Date".
Only include countries in Asia.
"""

csv_prompt = PromptTemplate.from_template(template)

csv_chain = csv_prompt | llm | csv_parser

input_data = {"question": "Give details of countries in Aisa"}

output = csv_chain.invoke(input_data)
output

['Afghanistan',
 'Kabul',
 'August 19',
 '1919\nArmenia',
 'Yerevan',
 'September 21',
 '1991\nAzerbaijan',
 'Baku',
 'October 18',
 '1991\nBahrain',
 'Manama',
 'December 16',
 '1971\nBangladesh',
 'Dhaka',
 'March 26',
 '1971\nBhutan',
 'Thimphu',
 'August 8',
 '1949\nBrunei',
 'Bandar Seri Begawan',
 'January 1',
 '1984\nCambodia',
 'Phnom Penh',
 'November 9',
 '1953\nChina',
 'Beijing',
 'October 1',
 '1949\nCyprus',
 'Nicosia',
 'October 1',
 '1960\nGeorgia',
 'Tbilisi',
 'April 9',
 '1991\nIndia',
 'New Delhi',
 'August 15',
 '1947\nIndonesia',
 'Jakarta',
 'August 17',
 '1945\nIran',
 'Tehran',
 'February 11',
 '1979\nIraq',
 'Baghdad',
 'October 3',
 '1932\nIsrael',
 'Jerusalem',
 'May 14',
 '1948\nJapan',
 'Tokyo',
 'February 11',
 '660 BC\nJordan',
 'Amman',
 'May 25',
 '1946\nKazakhstan',
 'Nur-Sultan',
 'December 16',
 '1991\nKuwait',
 'Kuwait City',
 'June 19',
 '1961\nKyrgyzstan',
 'Bishkek',
 'August 31',
 '1991\nLaos',
 'Vientiane',
 'July 19',
 '1949\nLebanon',
 'Beir

### Retrieval

In [92]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/arslanhaider/ahs-data/test-pdf.pdf")

pages = loader.load_and_split()

pages[0].page_content.split("\n")

['Welcome to Smallpdf',
 'Digital Documents—All In One Place',
 'Access Files Anytime, Anywhere Enhance Documents in One Click ',
 'Collaborate With Others With the new Smallpdf experience, you can ',
 'freely upload, organize, and share digital ',
 'documents. When you enable the ‘Storage’ ',
 'option, we’ll also store all processed files here. ',
 'You can access files stored on Smallpdf from ',
 'your computer, phone, or tablet. We’ll also ',
 'sync files from the Smallpdf Mobile App to our ',
 'online portalWhen you right-click on a file, we’ll present ',
 'you with an array of options to convert, ',
 'compress, or modify it. ',
 'Forget mundane administrative tasks. With ',
 'Smallpdf, you can request e-signatures, send ',
 'large files, or even enable the Smallpdf G Suite ',
 'App for your entire organization. Ready to take document management to the next level?']